In [12]:
import json
import rich
import copy
import sys
sys.path.append('../../')
from evaluate_utils import parse_predicted_moves, get_pydipcc_game

In [9]:
with open("ha_intent_prediction_parse.json", "r") as f:
    ha = json.load(f)

def extract_first_stage_results(data: str) -> str:
    """
    Extracts the results up to the first stage, discarding any results from subsequent stages.
    
    Parameters:
    - data: str - The input string containing the stages and their respective results.
    
    Returns:
    - str: The results up to the first stage.
    """
    stages = data.split('\n')
    result = []
    
    return stages[1].split(': ')[1]

def get_country(c):
    country_maps = {'A': 'AUSTRIA', 'E': 'ENGLAND', 'F': 'FRANCE', 'G': 'GERMANY', 'I': 'ITALY', 'R': 'RUSSIA', 'T': 'TURKEY'}
    return country_maps[c]

game = get_pydipcc_game("/data/user_data/wenkail/sotopia_diplomacy/clean_global/53480.json")

In [10]:
for h in ha:
    h['parse_predict_order_p1'] = parse_predicted_moves(game, get_country(h['power1']), h['predict_order_p1'], h['actual_order'])
    h['parse_predict_order_p2'] = parse_predicted_moves(game, get_country(h['power1']), h['predict_order_p2'], h['actual_order'])

In [11]:
with open("ha_intent_prediction_parse.json", "w") as f:
    json.dump(ha, f, indent=4)

### Data Clean


In [6]:
with open("human_annotation_intent_prediction.json", 'r') as f:
    ha = json.load(f)
for h in ha:
    h['predict_order_p1'] = extract_first_stage_results(h['predict_order_p1'])
    h['predict_order_p2'] = extract_first_stage_results(h['predict_order_p2'])

In [8]:
with open("ha_intent_prediction_parse.json", "w") as f:
    json.dump(ha, f, indent=4)

In [ ]:
# with open("/data/user_data/wenkail/sotopia_diplomacy/games/53480.json", 'r') as f:
#     game = json.load(f)
from tqdm import tqdm
import os

folder_path = "/data/user_data/wenkail/sotopia_diplomacy/games"
file_paths = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        file_paths.append(file_path)

games = []
for file_path in tqdm(file_paths):
    with open(file_path, 'r') as f:
        games.append(json.load(f))

for game in tqdm(games):
    for i in range(len(game['phases'])):
        if game["phases"][i]["messages"] != []:
            items = []
            for j, item in enumerate(game["phases"][i]['messages']):
                if item['recipient'] != "GLOBAL":
                    items.append(item)
                # else:
                    # print("Is Global")
            game["phases"][i]["messages"] = items


In [24]:
dir_path = "/data/user_data/wenkail/sotopia_diplomacy/clean_global/"
for game in tqdm(games):
    with open(dir_path + game['id']+".json", 'w') as f:
        json.dump(game, f, indent=4)

100%|██████████| 3047/3047 [01:27<00:00, 34.64it/s]


In [15]:
# Test GLOBAL
for game in tqdm(games):
    for phase in game['phases']:
        for message in phase['messages']:
            if message['recipient'] == "GLOBAL":
                print("Have Global")

100%|██████████| 3047/3047 [00:00<00:00, 6246.03it/s]


In [24]:
game['phases'][0]['messages']

[{'sender': 'AUSTRIA',
  'recipient': 'ENGLAND',
  'time_sent': 71,
  'phase': 'S1901M',
  'message': "Howdy.\n\nWe're on the opposite sides of the map, so there's not too much to talk about just yet, but it's always good to get a steady flow of information going. Shall we agree to share pertinent information whenever it becomes available?\r"},
 {'sender': 'AUSTRIA',
  'recipient': 'FRANCE',
  'time_sent': 208,
  'phase': 'S1901M',
  'message': "Hey France.\n\nOur only real mutual point of immediate interest right now is Italy, and given that one or both of us will inevitably be in a crush or be crushed scenario with him at some point, I think it's best to get a steady flow of information. I'll be in touch more once other people start talking, good luck to you.\r"},
 {'sender': 'AUSTRIA',
  'recipient': 'ITALY',
  'time_sent': 389,
  'phase': 'S1901M',
  'message': "Hey, Italy.\n\nObviously I'd like us to start positively, as the two of us fighting will just let Russia and Turkey grow,

In [51]:
with open("/data/user_data/wenkail/sotopia_diplomacy/clean_global/53480.json", "w") as f:
    json.dump(game, f, indent=4)

### Concat

In [1]:
import pandas as pd

In [6]:
ha = pd.read_csv("human_annotation.csv")
rd = pd.read_csv("/home/wenkail/diplomacy/sotopia-diplomacy/src/evaluate_intent/manual_annotations.csv")

In [9]:
rd.drop(columns=["upheld?", "betrayer"], inplace=True)

In [5]:
ha.drop(columns=["Unnamed: 1"])

,Unnamed: 0
0,1
1,2/3
2,NaN
3,0
4,1
...,...
69,0
70,2/3
71,1
72,0


In [21]:
result = pd.concat([rd, ha], axis=1)

In [23]:
result.drop(columns=["Unnamed: 1"], inplace=True)

In [25]:
result.dropna(inplace=True)

In [27]:
result

,gameID,phase,p1,p2,moves,Unnamed: 0
0,53480,S1901M,A,R,"A VIE - GAL, A WAR - GAL",1
1,53480,S1901M,E,G,"F LON - ENG, F EDI - NTH, A MUN - BUR",2/3
3,53480,S1901M,G,I,A MAR - PIE,0
4,53480,S1901M,R,T,"F ANK - BLA, F SEV - BLA",1
5,53480,F1901M,A,R,"F ALB - GRE, A SER S ALB - GRE, A VIE - GAL, A...",3/5
...,...,...,...,...,...,...
69,53480,F1908M,A,E,"F NAF - TUN, F ION S F NAF - TUN",0
70,53480,F1908M,A,I,"F WES S F ION - TUN, F ION - TUN, F TYS - LYO",2/3
71,53480,S1909M,E,R,"A RUH S A SIL - MUN, A SIL - MUN",1
72,53480,F1909M,A,R,"F RUM - SEV, A GAL - RUM",0


In [32]:
# mean_scores = 
result.groupby('phase')['Unnamed: 0'].mean()
# ['Unnamed: 0'].mean()

TypeError: agg function failed [how->mean,dtype->object]